#### Concepts: you can decorate methods. And classes!

Same thing but for classes. This introduces a new wrinkle when decorating methods.

The problem is that our decorator is invoked as each method is defined - we get
the function that results from the definition.

But to store that function `funct` we're going to need to have access to the
containing class to get to the `ENDPOINTS` class variable... and the class
hasn't been defined when our decorator runs!

We need to defer actually collecting the methods until after the class has been
defined - when it can be done by a class decorator!

Update the `endpoint` decorator to mark decorated methods and then update the
`collect` decorator to loop through the attributes of the class finding the
methods we marked and storing them in the `RemoteAPI.ENDPOINTS` dict.

Hint: help(vars). Also - functions are objects like anything else, open by
default so its totally fine to say `function._my_mark = 1`

In [1]:


def endpoint(f):
    f._endpoint = True # Modify the method (slightly)!
    return f


def collect(klass):
    for name, attr in vars(klass).items():
        if getattr(attr, "_endpoint", False):  # No idea what attr is. But does it have a ._endpoint
            klass.ENDPOINTS[name] = attr  # then stuff it in the dict as befor
    return klass

def default_function(klass):
    return "help"

@collect
class RemoteAPI:
    ENDPOINTS = {}

    def __init__(self, secrets=None):
        # Presumably load some secrets from a file so we can authenticate our client
        pass

    @endpoint
    def sessions(self, event_id=None):
        """Returns the session ids for the event."""
        return [1, 2, 3]

    @endpoint
    def events(self):
        """Returns the events to which you have access"""
        return [2717]

In [5]:
# Do not edit this cell
import ipywidgets as widgets
from IPython.display import display
inbox = widgets.Text(
    value='',
    placeholder='events or sessions',
    description='Input:',
    disabled=False   
)
button = widgets.Button(description='Click me',
    disabled=False,
    button_style='success', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check')
out = widgets.Output()
d = display(inbox, button, out)
def on_button_click(b):
    with out:
        # Using inspect to add flags for endpoint arguments ommitted for brevity
        def default_function(klass):
            print("Please specify a command.")
            print("command\t\tdescriptions")
            for (name, func) in klass.ENDPOINTS.items():
                print("\t".join([f"{name: <15}", func.__doc__]))
            return ""
        # Run command
        client = RemoteAPI()
        funct = client.ENDPOINTS.get(inbox.value.split()[0], default_function)
        print(funct(client))  # Passing the instance of RemoteAPI to self. Do you know why?

button.on_click(on_button_click)

Text(value='', description='Input:', placeholder='events or sessions')

Button(button_style='success', description='Click me', icon='check', style=ButtonStyle(), tooltip='Click me')

Output()